#**Text to Image Generation**

###**What Happens Step by Step?**
1. User Input  → Get text from the user to generate image
2. API Call → OpenAI DALL·E API returns a URL (not an image).
3. Download Image → requests.get(url).content fetches the image data.
4. Convert to Image → BytesIO turns the binary data into a format PIL can read.
5. Display in Gradio → The PIL image is returned and shown in the UI.


###**Install all dependencies**

In [3]:
!pip install openai gradio requests pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

### **Application**

######**Load Our Dataset**

In [2]:
!git clone https://github.com/cloudyuga/genai.git

Cloning into 'genai'...
remote: Enumerating objects: 1940, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 1940 (delta 53), reused 3 (delta 3), pack-reused 1840 (from 1)
Receiving objects: 100% (1940/1940), 5.35 MiB | 26.22 MiB/s, done.
Resolving deltas: 100% (966/966), done.


In [4]:
%cd genai
!git checkout hr

/content/genai
Branch 'hr' set up to track remote branch 'hr' from 'origin'.
Switched to a new branch 'hr'


In [5]:
!ls -R hrdataset/


hrdataset/:
employees  policies

hrdataset/employees:
101_Priya_Sharma.md  104_Karan_Kapoor.md   107_Amit_Verma.md	   110_Aditya_Jain.md
102_Rohit_Mehra.md   105_Sunita_Patil.md   108_Rajesh_Kulkarni.md  payroll_information.md
103_Anjali_Das.md    106_Neha_Malhotra.md  109_Meera_Iyer.md

hrdataset/policies:
employee_benefits.md  holiday_calendar.md  training_and_development.md
events_calendar.md    leave_policies.md


In [6]:
employee_files_path = "hrdataset/employees"
policy_files_path = "hrdataset/policies"


In [16]:
import os
import re

In [9]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [10]:
import gradio as gr
from openai import OpenAI

In [11]:
client = OpenAI(api_key=api_key)

In [64]:
import os
import re

# Function to read and extract details from Markdown files
def read_markdown_files(directory):
    """Read and parse content from all Markdown files in a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                content = f.read()

            # Extract details using regex
            employee_data = {
                "filename": filename,
                "Employee ID": re.search(r"Employee ID:\s*(\d+)", content),
                "Name": re.search(r"Name:\s*([\w\s]+)", content),
                "Role": re.search(r"Role:\s*([\w\s]+)", content),
                "Department": re.search(r"Department:\s*([\w\s]+)", content),
                "DOB": re.search(r"Date of Birth:\s*([\d-]+)", content),
                "Hobbies": re.search(r"Hobbies:\s*([\w\s,]+)", content)  # Extract hobbies
            }

            # Convert regex matches to actual values
            for key, match in employee_data.items():
                if match and hasattr(match, 'group'):  # Check if 'match' has 'group' attribute
                    employee_data[key] = match.group(1)  # Get the matched group value
                else:
                    employee_data[key] = None  # If no match or no 'group' method, set it to None

            documents.append(employee_data)

    return documents

In [65]:
# Load employee data from markdown files
employee_docs = read_markdown_files(employee_files_path)

In [66]:
# Function to extract Name and Role from the user's input using LLM
def extract_name_and_role(input_text):
    """Use LLM to extract Name and Role from user input."""
    try:
        response = client.Completion.create(
            model="text-davinci-003",
            prompt=f"Extract the Name and Role from this input: '{input_text}'.",
            max_tokens=100,
            temperature=0.3
        )
        extracted_text = response.choices[0].text.strip()
        # Extract Name and Role from the model's response
        name = re.search(r"Name: ([\w\s]+)", extracted_text)
        role = re.search(r"Role: ([\w\s]+)", extracted_text)

        return name.group(1) if name else None, role.group(1) if role else None
    except Exception as e:
        return None, None


In [67]:
# Function to fetch employee details from parsed docs
def get_employee_details(input_text, all_docs):
    input_text = input_text.lower()

    # Extract Name and Role from the input using LLM
    name, role = extract_name_and_role(input_text)

    # Search for employee details based on Name or Role
    for doc in all_docs:
        name_match = name and name.lower() in doc.get("Name", "").lower()
        role_match = role and role.lower() in doc.get("Role", "").lower()

        if name_match or role_match:
            return doc  # Return the matched employee details

    return None  # No match found


In [71]:
# Function to generate a professional birthday card
def generate_birthday_card(input_text):
    employee = get_employee_details(input_text, employee_docs)

    if employee:
        name = employee.get("Name", "Valued Employee")
        role = employee.get("Role", "Team Member")
        dob = employee.get("DOB", "N/A")
        hobbies = employee.get("Hobbies", "Enjoying life")  # Default if no hobby found

        # Crafting a professional, personalized birthday card prompt
        prompt = (
            f"A professional birthday card for {name}, our esteemed {role}. "
            f"Include the following details: Date of Birth: {dob}. "
            f"The card should be elegant and warm, with design elements related to {name}'s hobbies: {hobbies}. "
            f"Ensure the name {name} is clearly visible on the card."
        )
    else:
        prompt = f"A birthday card with vibrant colors and an artistic theme for {input_text}."

    try:
        # Generate image using DALL·E
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            n=1
        )
        return response.data[0].url  # Return the generated image URL

    except Exception as e:
        return f"Error: {str(e)}"

In [72]:
# Gradio Interface for generating birthday cards
iface = gr.Interface(
    fn=generate_birthday_card,
    inputs=gr.Textbox(
        lines=2,
        placeholder="Enter request (e.g., 'Generate a birthday card for CEO')",
        label="Card Request"
    ),
    outputs=gr.Image(label="Generated Card"),
    title="AI-Generated Birthday Card",
    description="Enter an employee name or role to generate a personalized birthday card."
)

# Launch Gradio App


In [73]:
if __name__ == "__main__":
    iface.launch()